In [1]:
from artifice.network import NN
from artifice.helper import apply_random_permutation, one_hot_encode, load_model

# Example on How to Build a Basic Neural Network:

In [61]:
from sympy import symbols, lambdify, Piecewise, Eq

In [68]:
class LossFunction:

    def __init__(self):
        
        # declare the variables of interest (x for activations, y and y_hat for loss)
        self.y_pred, self.y_true = symbols("y_pred y_true", real=True)
        self.expression = None

    def evaluate(self, y_pred, y_true, diff = False):

        assert self.expression is not None, "Loss function is not set!"

        if diff:
            self.func = lambdify((self.y_pred, self.y_true), self.expression.diff(self.y_pred))
        else:
            self.func = lambdify((self.y_pred, self.y_true), self.expression)

        return self.func(y_pred, y_true)


class MeanSquaredError(LossFunction):

    def __init__(self):
        super().__init__()

        self.expression = (self.y_pred - self.y_true)**2


    def __repr__(self):
        return f'Mean Squared Error Loss Function: \n{self.expression}'

# class MeanSquaredError(LossFunction):

#     def __init__(self):
#         super().__init__()

#         self.expression = (self.y_pred - self.y_true)**2

#         print(self.expression)


##########################################################################################

class ActivationFunction:

    def __init__(self):
        # declare the variables of interest (x for activations, y and y_hat for loss)
        self.x = symbols("x", real=True)
        self.expression = None

    def evaluate(self, x_val, diff = False):

        assert self.expression is not None, "Activation function is not set!"

        if diff:
            self.func = lambdify((self.x), self.expression.diff(self.x))
        else:
            self.func = lambdify((self.x), self.expression)

        return self.func(x_val)

class Relu(ActivationFunction):
    def __init__(self):

        super().__init__()

        self.expression = Piecewise((0,self.x<0),(self.x, self.x>=0))

    def __repr__(self):
        return f'Relu Activation Function: \n{self.expression}'


In [69]:
MeanSquaredError()#.evaluate(1, 2, diff=False)

Mean Squared Error Loss Function: 
(y_pred - y_true)**2

In [71]:
Relu()

Piecewise((0, x < 0), (x, True))


Relu Activation Function: 
Piecewise((0, x < 0), (x, True))

In [4]:
""" Code written by William Bidle and Ilana Zane """

__version__ = "dev"

import os
import json
import pickle
from typing import List, Union, Tuple

import numpy as np
import matplotlib.pyplot as plt
import sympy
from tqdm import tqdm


class NN:

    """
    Defines a neural network.

    :param layer_sequence: A list containing the nodes per layer and correcponding activation
    functions between layers.
    :param loss_function: The desired loss function to be used.
    """

    def __init__(self, layer_sequence: list = None, loss_function: str = "MSE"):

        # Load in the function library
        self.activation_funcs_library = self.__load_func_libraries(
            "activation_funcs_library.txt"
        )

        # Separate out the layer information (every other element)
        layers = layer_sequence[::2]

        # check that the declared number of nodes is an integer
        if all(isinstance(item, int) for item in layers) is True:
            # set the 'layers' property
            layers = np.array(layer_sequence[::2], dtype=int)

            # initialize the *weights* property based off of the desired layer sequence
            self.weights = self.__initialize_weights(layers)

        else:
            # raise an exception if the input layer sequence is improperly defined
            raise ValueError("Invalid Layer Sequence!")

        # separate out the activation function information
        activation_funcs = layer_sequence[1::2]

        # check that the activation functions are strings
        if all(isinstance(item, str) for item in activation_funcs) is True:
            # initialize the 'activation_funcs' property
            self.activation_funcs = []

            # initialize each declared activation functions between layers
            for activation_func in activation_funcs:
                self.activation_funcs.append(
                    self.__init_func(self.activation_funcs_library, activation_func)
                )

        else:
            # raise an exception if the input layer sequence is improperly defined
            raise ValueError("Invalid Layer Sequence!")

        # initialize the *loss_funcs_library* property
        self.loss_funcs_library = self.__load_func_libraries("loss_funcs_library.txt")

        # initialize the *loss_func* property
        self.loss_func = self.__init_func(self.loss_funcs_library, loss_function)

        # initialize the *loss_func_label* property (used in plotting for now)
        self.loss_func_label = loss_function

        # initialize the *training_err* property (will be set later once the model is trained)
        self.training_err = None

    def __load_func_libraries(self, func_file: str) -> dict:

        """
        Loads in dictionaries of available functions.

        :param func_file: the filename containing the library of usable functions.
        :returns func_library: a dictionary of the usable functions.
        """

        # open the desired file
        with open(
            os.path.join(os.path.dirname(__file__), func_file), encoding="utf-8"
        ) as f:
            data = f.read()

        # reconstruct the data as a dictionary
        func_library = json.loads(data)

        return func_library

    def __init_func(
        self, func_library: dict, func_name: str
    ) -> sympy.core.symbol.Symbol:

        """
        Initialize a function from a function library

        :param func_library: A dictionary of the usable functions.
        :param func_name: The name of the mathematical function to be initialized (e.g., 'sigmoid').
        :returns expression: Symbolic mathematical representation of 'func_name'.
        """

        # try to initialize the function
        try:
            expression = func_library[func_name]

        # if the function doesn't exist within the function library, return an exception
        except Exception as exc:
            raise ValueError(
                f"Desired function '{func_name}' does not exist within the 'func_library.'"
            ) from exc

        # declare the variables of interest (x for activations, y and y_hat for loss)
        x, y, y_hat = sympy.symbols("x y y_hat", real=True)

        # parse throught the expression
        expression = sympy.parse_expr(
            expression, local_dict={"x": x, "y": y, "y_hat": y_hat}
        )

        return expression

    def __eval_func(
        self, expression: sympy.core.symbol.Symbol, vals: List[List], diff: bool = False
    ) -> Union[float, np.ndarray]:

        """
        Initialize an activation function.

        :param expression: Symbolic mathematical representation of 'func'.
        :param vals: The values to evaluate 'expression' with - 1 sub-list for activation functions
        (x information), 2 sub-lists for loss functions (y, y_hat information).
        :param diff: Whether or not to evaluate the derivitive of 'expression' at 'vals'.
        :returns result: evaluation of 'expression' at '_input_' - if diff = False -> Float,
        if diff = True -> np.ndarray.
        """

        # Evaluate Activation Functions
        if expression in self.activation_funcs:

            # the variable of interest
            x = sympy.Symbol("x", real=True)

            # differentiate only if the 'diff' flag is True
            if diff is True:
                expression = expression.diff(x)

            # allow the function to be evaluated from lists
            func = sympy.lambdify(x, expression)

            # evaluate the function at the given input
            result = func(vals[0])

        # Evaluate Loss Functions
        else:

            # the variables of interest
            y, y_hat = sympy.symbols("y, y_hat", real=True)

            # differentiate only if the 'diff' flag is True
            if diff is True:
                expression = expression.diff(y)

                # allow the function to be evaluated from lists
                func = sympy.lambdify((y, y_hat), expression)

                # evaluate the function at the given input
                result = func(vals[0], vals[1])

            else:

                # allow the function to be evaluated from lists
                func = sympy.lambdify((y, y_hat), expression)

                # evaluate the function at the given input
                result = sum(func(vals[0], vals[1]))

        return result

    def __initialize_weights(self, layers: np.ndarray) -> List[np.ndarray]:

        """
        Initialize the weights of the network.

        :params layers: An array containing the layer information of the network.
        :returns weights: List containing the 2D weight arrays between the different layers.
        """

        layers_reorganized = np.flip(
            layers.repeat(2)[1:-1].reshape(len(layers) - 1, 2), axis=1
        )

        # initialize the list of the weights between different layers
        weights = []

        for layer_reorganized in layers_reorganized:
            # include bias vector with the '+ 1'
            weight = np.random.randn(layer_reorganized[0], layer_reorganized[1] + 1)

            # HE initialization for weights
            weights.append(weight * np.sqrt(2 / layer_reorganized[1]))

        return weights

    def __update_weights(
        self, weights: List[np.ndarray], layer_values: List[List], _label_: np.ndarray
    ) -> Tuple[List[np.ndarray], List[np.ndarray]]:

        """
        Update the weights of the network.

        :params weights: List containing the 2D weight arrays between the different layers.
        :params layer_values: List containing the values of each layer for a given input value.
        :params _label_: Array representation for the current label, usually one hot ecoded.
        :returns weight_updates: List of updated 2D weight arrays between the different layers.
        :returns weights: List of original 2D weight arrays between the different layers.
        """

        # get the list of desired activation functions
        activations = self.activation_funcs

        # make a copy of the weights so they aren't changed
        weight_updates = weights.copy()

        # blue in notes
        blue = np.diag(
            self.__eval_func(self.loss_func, [layer_values[-1], _label_], diff=True)
        )

        # need to add an extra component to input for bias
        layer_output = np.dot(weights[-1], np.concatenate((layer_values[-2], [1])))

        # red in notes
        red = self.__eval_func(activations[-1], [layer_output], diff=True)

        # index through each weight (work backwards)
        for i in range(len(weights), 0, -1):

            # pink in notes
            pink = np.concatenate((layer_values[i - 1], [1]))

            # first two terms in gradient
            grad = np.matmul(blue, np.outer(red, pink))

            # look forwards through each weight (only if there are forward weights)
            for j in range(len(weights), i, -1):

                # orange in notes
                orange = np.transpose(weights[j - 1])

                # green in notes
                green = np.diag(
                    self.__eval_func(
                        activations[j - 1],
                        [
                            np.dot(
                                weights[j - 2],
                                np.concatenate((layer_values[j - 2], [1])),
                            )
                        ],
                        diff=True,
                    )
                )

                # add on the bias vector to make sure dimensions work properly
                bias_vec = np.ones((len(green), 1))

                # incorperate the bias
                green = np.hstack((green, bias_vec))

                # now multiply the rest to grad
                grad = np.matmul(green, np.matmul(orange, grad))

            # record the change in weight
            weight_updates[i - 1] = grad

        return weight_updates, weights

    def get_network_outputs(
        self, weights: List[np.ndarray], _input_: np.ndarray
    ) -> np.ndarray:
        """
        Initialize the weights of the network.

        :params weights: List containing the 2D weight arrays between the different layers.
        :params _inputs_: Input layer to the network.
        :returns network_outputs: Output layer of the network.
        """

        # get the list of desired activation functions
        activations = self.activation_funcs

        # add the first layer to the list
        current_layer = _input_
        network_outputs = [current_layer]

        for index, weight in enumerate(weights):

            # need to add an extra component to input for bias
            layer_output = np.dot(weight, np.concatenate((current_layer, [1])))

            current_layer = self.__eval_func(
                activations[index], [layer_output], diff=False
            )

            network_outputs.append(current_layer)

        return network_outputs

    def compute_error(self, _result_: np.ndarray, _label_: np.ndarray) -> float:
        """
        Compute the error between the neural network's output and expected value.

        :params _result_: Output layer of the network.
        :params _label_: Expected result.
        :returns error: The error of the network.
        """
        error = self.__eval_func(self.loss_func, [_result_, _label_])

        return error

    def train(  # pylint: disable=too-many-arguments, too-many-locals
        self, x_train, y_train, batch_size=1, epochs=1, epsilon=1, visualize=False
    ) -> None:
        """
        Train a model.

        :params x_train:
        :params y_train:
        :params batch_size:
        :params epochs:
        :params epsilon:
        :params visualize:
        """

        weights = self.weights  # get the list of weights

        error_list = []

        counter = 0  # keep track of the current iteration

        weights_list = (
            {}
        )  # create a dictionary to keep track of the weight updates (batch size)

        # just a temporary blank array since training hasn't begun yet
        for i in range(len(weights)):
            weights_list[i] = []

        for i in range(epochs):

            # iterate through the inputs and labels
            for _input_, _label_ in tqdm(
                zip(x_train, y_train), total=len(x_train), desc=f"Epoch {str(i + 1)}"
            ):

                network_output = self.get_network_outputs(
                    weights, _input_
                )  # the current network output

                error = self.compute_error(network_output[-1], _label_)

                weight_updates, weights = self.__update_weights(
                    weights, network_output, _label_
                )

                for j in range(len(weights)):
                    weights_list[j].append(weight_updates[j])

                counter += 1

                if (counter) % batch_size == 0:
                    for index, weight in enumerate(weights):

                        weights[index] = weight - epsilon * np.average(
                            np.array(weights_list[index]), axis=0
                        )
                        weights_list[index] = []

                error_list.append(error)

            self.weights = weights
            self.training_err = error_list

        if visualize is True:
            _, ax = plt.subplots(figsize=(12, 6))

            ax.plot(self.training_err)  # to visualize error over time

            ax.set_xlabel("Training Sample", fontsize=14)
            ax.set_ylabel(f"{self.loss_func_label} Error", fontsize=14)

            ax.grid(linestyle="--")

            plt.show()

    def evaluate(self, x_test):
        """
        Evaluate a model.

        :params x_test:
        :returns results:
        """

        results = []

        for _input_ in tqdm(x_test, desc="Evaluating Test Data", total=len(x_test)):
            # the current network output
            network_output = self.get_network_outputs(self.weights, _input_)
            results.append(network_output[-1])

        return results

    def save_model(self, out_dir: str, filename: str) -> None:
        """
        Save a model.

        :params filename:
        """

        # Enforce trailing backslash to directory
        out_dir = os.path.join(out_dir, "")

        # Check if out_dir exists
        if not os.path.exists(out_dir):
            raise ValueError(f"Invalid path: {out_dir}.")

        save_path = out_dir + filename

        to_save = [
            self.weights,
            self.activation_funcs,
        ]  # save both the activations and weights
        with open(save_path, "wb") as fp:  # save the weights and activations
            pickle.dump(to_save, fp)

        print()
        print(f"Model saved at '{save_path}'")
        print()


In [5]:
layer_sequence = [1,'ReLU', 2, 'sigmoid', 3]
loss_function = 'MSLE'

nn = NN(layer_sequence, loss_function)

# Avaliable Class Properties
print('activation func library:\n', nn.activation_funcs_library, '\n')
print('loss func library:\n', nn.loss_funcs_library, '\n')
print('current weights:\n', nn.weights, '\n')
print('current activation functions:\n', nn.activation_funcs, '\n')
print('current loss function:\n', nn.loss_func_label, ':', nn.loss_func, '\n')
print('traing error:\n', nn.training_err, '\n')

NameError: name '__file__' is not defined

# Example of Classification with MNIST Data:

In [ ]:
''' For Dataset Usage '''
from tensorflow.keras.datasets import mnist

In [ ]:
########################### Training Data Prep ###########################

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = (x_train.astype('float32') / 255.).reshape(len(x_train), len(x_train[0])*len(x_train[0][0]))
x_test = (x_test.astype('float32') / 255.).reshape(len(x_test), len(x_test[0])*len(x_test[0][0]))

num_data_pts = 1000

x_train = x_train[0:num_data_pts]
y_train = one_hot_encode(y_train[0:num_data_pts])

input_shape = x_train[0].size
output_shape = y_train[0].size

########################### Testing Data Prep ############################

num_test_evals = 1000

x_test = x_test[0:num_test_evals]
y_test = one_hot_encode(y_test[0:num_test_evals])

##########################################################################

In [ ]:
############################## Build network ##############################

# layer_sequence = [input_shape, 'ReLU', 392, 'ReLU', 196, 'ReLU', 98, 'ReLU', 49, 'sigmoid', output_shape]
layer_sequence = [input_shape, 'ReLU', 100, 'sigmoid', output_shape] # initialize the layer sequences and corresponding activations

loss_function = 'MSE' # declare the loss function

nn = NN(layer_sequence, loss_function) # Build a model 

################################# Run it ##################################

nn.train(x_train, y_train, epochs = 1, batch_size = 5, epsilon = 0.01, visualize = True)

################################# Testing ##################################

nn.evaluate(x_test)

nn.save_model(out_dir="../../temp/saved_models", filename="model")

# Comparison of Different Loss Functions:

In [ ]:
########################### Training Data Prep ###########################

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = (x_train.astype('float32') / 255.).reshape(len(x_train), len(x_train[0])*len(x_train[0][0]))
x_test = (x_test.astype('float32') / 255.).reshape(len(x_test), len(x_test[0])*len(x_test[0][0]))

num_data_pts = 1000

x_train = x_train[0:num_data_pts]
y_train = One_Hot_Encode(y_train[0:num_data_pts])

input_shape = x_train[0].size
output_shape = y_train[0].size

########################### Testing Data Prep ############################

num_test_evals = 1000

x_test = x_test[0:num_test_evals]
y_test = One_Hot_Encode(y_test[0:num_test_evals])

##########################################################################

In [ ]:
# layer_sequence = [input_shape, 'ReLU', 392, 'ReLU', 196, 'ReLU', 98, 'ReLU', 49, 'sigmoid', output_shape]
layer_sequence = [input_shape, 'ReLU', 100, 'sigmoid', output_shape] # initialize the layer sequences and corresponding activations


##########################################################################
############################## Build network #############################
##########################################################################
loss_function = 'MSE'

nn1 = NN(layer_sequence, loss_function) # Build model 

################################# Run it ##################################

nn1.train(x_train, y_train, batch_size = 5, epsilon = 0.01, visualize = False)

##########################################################################
##########################################################################
##########################################################################

##########################################################################
############################## Build network #############################
##########################################################################

loss_function = 'MAE'

nn2 = NN(layer_sequence, loss_function) # Build model 

################################# Run it ##################################

nn2.train(x_train, y_train, batch_size = 5, epsilon = 0.01, visualize = False)

##########################################################################
##########################################################################
##########################################################################

##########################################################################
############################## Build network #############################
##########################################################################

loss_function = 'MAPE'

nn3 = NN(layer_sequence, loss_function) # Build model 

################################# Run it ##################################

nn3.train(x_train, y_train, batch_size = 5, epsilon = 0.01, visualize = False)

##########################################################################
##########################################################################
##########################################################################

##########################################################################
############################## Build network #############################
##########################################################################

loss_function = 'MSLE'

nn4 = NN(layer_sequence, loss_function) # Build model 

################################# Run it ##################################

nn4.train(x_train, y_train, batch_size = 5, epsilon = 0.01, visualize = False)

##########################################################################
##########################################################################
##########################################################################

In [ ]:
fig, ax = plt.subplots(2,2, figsize = (16,8))
            
ax[0][0].plot(nn1.training_err, label = '%s' % str(nn1.loss_func)) # to visualize error over time
ax[0][0].set_xlabel('Training Sample', fontsize = 14)
ax[0][0].set_ylabel('%s Error' % nn1.loss_func_label, fontsize = 14)
ax[0][0].grid(linestyle = '--')
ax[0][0].legend(fontsize = 12)

ax[0][1].plot(nn2.training_err, label = '%s' % str(nn2.loss_func)) # to visualize error over time
ax[0][1].set_xlabel('Training Sample', fontsize = 14)
ax[0][1].set_ylabel('%s Error' % nn2.loss_func_label, fontsize = 14)
ax[0][1].grid(linestyle = '--')
ax[0][1].legend(fontsize = 12)

ax[1][0].plot(nn3.training_err, label = '%s' % str(nn3.loss_func)) # to visualize error over time
ax[1][0].set_xlabel('Training Sample', fontsize = 14)
ax[1][0].set_ylabel('%s Error' % nn3.loss_func_label, fontsize = 14)
ax[1][0].grid(linestyle = '--')
ax[1][0].legend(fontsize = 12)

ax[1][1].plot(nn4.training_err, label = '%s' % str(nn4.loss_func)) # to visualize error over time
ax[1][1].set_xlabel('Training Sample', fontsize = 14)
ax[1][1].set_ylabel('%s Error' % nn4.loss_func_label, fontsize = 14)
ax[1][1].grid(linestyle = '--')
ax[1][1].legend(fontsize = 12)

plt.show()

# Example of Classification with MNIST (only 2 labels):

In [ ]:
first_number = 0
second_number = 1

################################# Params #################################
(x_train, y_train), (x_test, y_test) = mnist.load_data()

train_zeros = y_train == first_number
train_ones = y_train == second_number

x_train = np.array(list(x_train[train_zeros]) + list(x_train[train_ones]))
y_train = np.array(list(y_train[train_zeros]) + list(y_train[train_ones]))

x_train = (x_train.astype('float32') / 255.).reshape(len(x_train), len(x_train[0])*len(x_train[0][0]))

x_train, y_train = unison_shuffled_copies(x_train, y_train)

x_train, y_train = x_train[:1000], One_Hot_Encode(y_train[:1000])

test_zeros = y_test == first_number
test_ones = y_test == second_number

x_test = np.array(list(x_test[test_zeros]) + list(x_test[test_ones]))
y_test = np.array(list(y_test[test_zeros]) + list(y_test[test_ones]))

x_test = (x_test.astype('float32') / 255.).reshape(len(x_test), len(x_test[0])*len(x_test[0][0]))

x_test, y_test = unison_shuffled_copies(x_test, y_test)

x_test, y_test = x_test[:1000], One_Hot_Encode(y_test[:1000])

input_shape = x_train[0].size
output_shape = y_train[0].size


############################## Build network ##############################

# layer_sequence = [input_shape, 'ReLU', 392, 'ReLU', 196, 'ReLU', 98, 'ReLU', 49, 'sigmoid', output_shape]
layer_sequence = [input_shape, 'ReLU', 100, 'sigmoid', output_shape]

loss_function = 'MSE'
# loss_function = 'Binary Cross-Entropy'

nn = NN(layer_sequence, loss_function) # Build model 

################################# Run it ##################################

nn.train(x_train, y_train, batch_size = 5, epsilon = 0.1, visualize = True)

################################# Testing ##################################

nn.evaluate(x_test, y_test)

# Compare to TensorFlow

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

In [ ]:
# mnist = tf.keras.datasets.mnist

# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0



(x_train, y_train), (x_test, y_test) = mnist.load_data()


zeros_train = y_train == 0
x_train_zeros, y_train_zeros = list(x_train[zeros_train]), list(y_train[zeros_train])
zeros_test = y_test == 0
x_test_zeros, y_test_zeros = list(x_test[zeros_test]), list(y_test[zeros_test])

ones_train = y_train == 1
x_train_ones, y_train_ones = list(x_train[ones_train]), list(y_train[ones_train])
ones_test = y_test == 1
x_test_ones, y_test_ones = list(x_test[ones_test]), list(y_test[ones_test])


x_train, y_train = np.array(x_train_zeros + x_train_ones), np.array(y_train_zeros + y_train_ones)
x_test, y_test = np.array(x_test_zeros + x_test_ones), np.array(y_test_zeros + y_test_ones)


# shuffle arrays

x_train, y_train = unison_shuffled_copies(x_train, y_train)
x_test, y_test = unison_shuffled_copies(x_test, y_test)

x_train = (x_train / 255.)
x_test = (x_test / 255.)

print(x_train.shape)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(100, activation='relu'),
  #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2, activation='sigmoid')
])
loss_fn = tf.keras.losses.MeanSquaredError()

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, batch_size= 5, epochs=1)